<a href="https://colab.research.google.com/github/dean-baldwin/linkedin_html_scraper/blob/main/LinkedIn_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LinkedIn Post Scraper
**Instructions**
1. Go to LinkedIn `Post` page that you want to scrape.
2. Be sure to click `Read More...` on all the targeted posts.
3. Right click and select `Inspect` copy all of the outer `<html>` code.
4. Upload it to Colab.
5. Be sure to change the file path and set the output name for the csv.file.


**Install necessary libraries**

In [ ]:
pip install selenium

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import re as re
import time
import pandas as pd

**USER ACTION: Set the path to the html file**

In [5]:
with open("/content/lallemand.html") as fp:
    company_page = bs(fp, 'html.parser')

In [6]:
linkedin_soup = bs(company_page.encode("utf-8"), "html")
linkedin_soup.prettify()

#Find the post blocks
containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})

In [ ]:
# Lists that we will iterate to
post_dates = []
post_texts = []
post_likes = []
post_comments = []
video_views = []
media_links = []
media_type = []

post_subtexts = []


#Looping through the posts and appending them to the lists
for container in containers:

  #Try function to make sure its a post and not a promotion
  try:
    posted_date = container.find("span",{"class":"visually-hidden"})
    text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper"})
    text = text_box.find("span",{"dir":"ltr"})
    new_likes = container.findAll("span", {"class":"social-details-social-counts__reactions-count"})
    new_comments = container.findAll("span", {"class": "social-details-social-counts__item-text--with-social-proof"})

    # begin subtext
    '''
    text_box2 = container.find("div",{"class":"feed-shared-mini-update-v2 feed-shared-update-v2__update-content-wrapper artdeco-card"})
    text2 = text_box2.find("div",{"class":"feed-shared-text relative feed-shared-update-v2__commentary "})

    post_subtexts.append(text2.text.strip())
    print(post_subtexts)
    '''
    # end subtext

    #Appending date and text to lists
    post_dates.append(posted_date.text.strip())
    post_texts.append(text_box.text.strip())




    #Determining media type and collecting relevant info for each type
    try:
      video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
      video_link = video_box[0].find("video", {"class":"vjs-tech"})
      media_links.append(video_link['src'])
      media_type.append("Video")
    except:
      try:
        image_box = container.findAll("div",{"class": "feed-shared-image__container"})
        image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
        media_links.append(image_link['src'])
        media_type.append("Image")
      except:
        try:
          image_box = container.findAll("div",{"class": "feed-shared-image__container"})
          image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
          media_links.append(image_link['src'])
          media_type.append("Image")
        except:
          try:
            article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
            article_link = article_box[0].find('a', href=True)
            media_links.append(article_link['href'])
            media_type.append("Article")
          except:
            try:
              video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
              video_link = video_box[0].find('a', href=True)
              media_links.append(video_link['href'])
              media_type.append("Youtube Video")   
            except:
              try:
                poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                media_links.append("None")
                media_type.append("Other: Poll, Shared Post, etc")
              except:
                media_links.append("None")
                media_type.append("Unknown")



    #Getting Video Views. (The folling three lines prevents class name overlap)
    view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
    view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
    result = view_container2 - view_container1

    view_container = []
    for i in result:
      view_container += i

    try:
      video_views.append(view_container[1].text.strip().replace(' Views',''))

    except:
      video_views.append('N/A')

    
    #Appending likes and comments if they exist
    try:
      post_likes.append(new_likes[0].text.strip())
    except:
      post_likes.append(0)
      pass

    try:
      post_comments.append(new_comments[0].text.strip().replace('comment',''))                           
    except:                                                           
      post_comments.append(0)
      pass

  except:
      pass

In [64]:
#Stripping non-numeric values
comment_count = []
for i in post_comments:
    s = str(i).replace('Comment','').replace('s','').replace(' ','')
    comment_count += [s]

**USER ACTION: Set the name of the .csv file**

In [65]:
data = {
    "Date Posted": post_dates,
    "Media Type": media_type,
    "Post Text": post_texts,
    #"Post Subtext": post_subtexts,
    "Post Likes": post_likes,
    "Post Comments": comment_count,
    "Video Views": video_views,
    "Media Links": media_links
}

df = pd.DataFrame(data)


#Exporting as csv file to program folder
df.to_csv("linkedin_posts.csv", encoding='utf-8', index=False)
